In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import json
from collections import defaultdict
from matplotlib import pyplot as plt
import cv2
import numpy as np
import pandas as pd
from skimage import io
from PIL import Image
from tqdm import tqdm
Image.MAX_IMAGE_PIXELS = None

In [ ]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility/src')
sys.path.append(PATH)

In [ ]:
from lib.utilities_alignment import (create_warp_transforms, parse_elastix)

In [ ]:
animal = 'DK55'
downsample_factor = 32
CSV_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/{animal}'

In [ ]:
IMG_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'
cshl_csvfile = 'cshl.premotor.csv'
cshl_csvpath = os.path.join(CSV_PATH, cshl_csvfile)
cshl_df = pd.read_csv(cshl_csvpath, names=['section','x','y'])

In [ ]:
sections = cshl_df['section'].unique().tolist()

In [ ]:
section_size = {}
for section in sections:
    filename = str(section).zfill(3) + '.tif'
    filepath = os.path.join(IMG_PATH, 'CH3/full', filename)
    input_image = Image.open(filepath)
    rotated_height = input_image.width
    section_size[section] = rotated_height
    input_image.close()

In [ ]:
for index, row in cshl_df.iterrows():
    section = row['section']
    rotated_height = section_size[section]
    cshl_df.at[index,'xp'] = cshl_df.at[index,'y'] 
    cshl_df.at[index,'yp'] = rotated_height - cshl_df.at[index,'x'] 

In [ ]:
cshl_df.head()

## Get the annotation points

In [ ]:
section_vertices = defaultdict(list)
for index, row in cshl_df.iterrows():
    section = row['section']
    x = row['xp']
    y = row['yp']
    section_vertices[section].append([x,y])

## Reproduce create_clean transform

In [ ]:
# find the difference between the image stack size which is consistent for all images
# and the individual shape of the aligned image: use thumbnail 
INPUT = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH3/thumbnail'
section_offset = {}

# original image is long and narrow with the brain stem at the bottom
width = 34000
height = 60000
downsample_factor = 32
# these get switched here
aligned_shape = np.array((width, height)) / downsample_factor
print(aligned_shape)

for file_name in sorted(os.listdir(INPUT)):
    filepath = os.path.join(INPUT, file_name)    
    input_image = Image.open(filepath)
    width = input_image.width
    height = input_image.height
    downsampled_shape = np.array((width, height))
    difference = np.round(aligned_shape - downsampled_shape)
    section = int(file_name.split('.')[0])
    #print(section, difference, width,height)
    section_offset[section] = difference / 2


In [ ]:
print(section, downsample_factor, section_offset[224])
#section_offset[224] = [68,66]

## Reproduce create_alignment transform

In [ ]:
transforms = parse_elastix(animal)
downsample_factor = 32
warp_transforms = create_warp_transforms(animal, transforms, downsample=True)
ordered_transforms = sorted(warp_transforms.items())

section_transform = {}
for filename, transform in ordered_transforms:
    section_num = int(filename.split('.')[0])
    transform = np.linalg.inv(transform) keeping this makes it worse
    section_transform[section_num] = transform

## Alignment of annotation coordinates

In [ ]:
'''
(x', y') = (x * sx + y * ry + tx, x * rx + y * sy + ty)
'sx': T[0, 0], 'sy': T[1, 1], 'rx': T[1, 0], 'ry': T[0, 1], 'tx': T[0, 2], 'ty': T[1, 2]
'''
def transform_create_alignment(points, transform):
    a = np.hstack((points, np.ones((points.shape[0], 1))))
    b = transform.T[:, 0:2]
    c = np.matmul(a, b)
    return c

aligned_section_structure_polygons = defaultdict(list)
for section, vertices in section_vertices.items():
    points = np.array(vertices)
    points = points + section_offset[section] # create_clean offset
    points = transform_create_alignment(points, section_transform[section]) # create_alignment transform
    aligned_section_structure_polygons[section] = [points]

In [ ]:
data = []
for section,v in aligned_section_structure_polygons.items():
    for x,y in v[0]:
        data.append([x,y,section])    

df = pd.DataFrame(data, columns=['x','y','section'])
df = df.astype({'section':'int32','x': 'float64', 'y':'float64'})
outfile = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/DK55/cshl2dk.aligned.csv'
#df.to_csv(outfile, index=False, header=False)
df.head()

To this point, aligned_section_structure_polygons variable contains the aligned polygon vertices for each structure in each section. 
From now on, we introduce how to draw these points to numpy array or neuroglancer

In [ ]:
section = 224
filename = f'{str(section).zfill(3)}.tif'
INPUT = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK55/preps/CH3'
outpath = os.path.join(INPUT, '224.out.noinv.switchedoffsets.tif')
filepath = os.path.join(INPUT, 'thumbnail_aligned_norm', filename)
#filepath = os.path.join(INPUT, '224.norm.rotated.tif')
img = cv2.imread(filepath, -1)
#img = (img/256).astype(np.uint8)
#clahe = cv2.createCLAHE(clipLimit=30.0, tileGridSize=(4, 4))
#img = clahe.apply(img)

radius = 5
color = (0,255,1)
#df = cshl_df.copy()
df = df.loc[df['section'] == section]
for index, row in df.iterrows():
    x = round(row['x']/32) 
    y = round(row['y']/32)
    #print(x,y)
    cv2.circle(img, (int(x), int(y)), radius, color, 2)

cv2.imwrite(outpath, img)
fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(img, cmap="gray")
plt.title('Aligned section:{}'.format(section), fontsize=30)
plt.tick_params(axis='x', labelsize=30)
plt.tick_params(axis='y', labelsize=30)
plt.show()